In [0]:
%reload_ext autotime
FORECAST_TERM_CODE = 202508
LAG = 1
exec(open('./amp.py').read())
from sklearn.metrics import RocCurveDisplay
from sklearn.calibration import CalibrationDisplay

aggregates = union(
    'styp_desc','camp_desc',
    # 'coll_desc','dept_desc','majr_desc','hs_qrtl','tsi_math','tsi_reading','tsi_writing','resd_desc','oriented','waiver','lgcy',
    # 'gender',races,'international',
)

features = {'act_equiv':pd.NA,'age':pd.NA,'camp_desc':'stephenville','drivetime':pd.NA,'eager':pd.NA,'fafsa': False,'finaid': False,'gap_score':0,'gender':pd.NA,'hs_qrtl':pd.NA,'international':False,'lgcy':False,'oriented':False,**{r: False for r in races},'schlship':False,'ssb':False,'styp_desc':'new first time','_tot_sch':0,'tsi_math':False,'tsi_reading':False,'tsi_writing':False,'verified':False,'waiver':False}

crse_codes = union('_headcnt','_proba',
    'agec2317','ansc1119','ansc1319','anth2302','anth2351','arts1301','arts1303','arts1304','arts3331','biol1305','biol1406','biol1407','biol2401','biol2402','busi1301','busi1307','chem1111','chem1112','chem1302','chem1311','chem1312','chem1407','chem1409','cnst1301','comm1311','comm1315','comm2302','crij1301','dram1310','dram2361','easc2310','econ1301','econ2301','educ1301','engl1301','engl1302','engl2307','engl2320','engl2321','engl2326','engl2340','engl2350','engl2360','engl2362','engl2364','engl2366','engl2368','engr1211','engr2303','envs1302','fina1360','geog1303','geog1320','geog1451','geog2301','geol1403','geol1404','geol1407','geol1408','govt2305','govt2306','hist1301','hist1302','hist2321','hist2322','huma1315','kine1301','kine1338','kine2315','math1314','math1316','math1324','math1325','math1332','math1342','math1352','math2412','math2413','musi1303','musi1310','musi1311','musi2350','musi3325','phil1301','phil1304','phil2303','phil3301','phys1302','phys1401','phys1402','phys1403','phys1411','phys2425','phys2426','psyc2301','psyc3303','psyc3307','soci1301','soci1306','soci2303','univ0010','univ0200','univ0204','univ0301','univ0314','univ0324','univ0332','univ0342',)

REFRESHED = []
REFRESH = union(
    # 'flags',
    # 'admissions',
    # 'registrations',
    # 'students',
    # 'enrollments',
    # 'imputed',
    # 'learners',
    # 'predictions',
    # 'studentsall',
    # 'enrollmentsall',
    # 'predictionsall',
    # 'proba',
    # 'forecasts',
)

kwargs = dict(features=features, aggregates=aggregates, crse_codes=crse_codes, subpops='styp_desc', start=2022,
    # date='2025-10-30',
    flaml=dict(
        time_budget=60,
    ),)
if LAG > 0: #retention
    kwargs |= dict(crse_codes='_proba', aggregates=[])
kwargs |= dict(crse_codes=['_headcnt','_proba'], aggregates='camp_desc')  #quick for testing
self = AMP(**kwargs)
# self.get_flagsday(); # GA's should not have permissions to run this because it sees pii
self.get_proba();
# self.get_reports();   # may hit memory limits running get_flagsyear - just restart kernel and try again

In [0]:
def run_viz():
    A = self.get_proba().query(f'cohort_term != @self.term_desc & _tot_sch > 0').sort_values(['cohort_term', 'model_term','prediction'])
    sns.displot(data=A, x='prediction', hue='actual', kind='kde', row='cohort_term', col='model_term', common_norm=False)
    plt.show()

    G = A.groupby('cohort_term')
    nrows, ncols, length = G.ngroups, 2, 5
    fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols*length, nrows*length))
    for i, (cohort_term, B) in enumerate(G):
        def fmt(nm):
            # ax.plot([0, 1], [0, 1], 'k--')
            ax.set_title(f'{cohort_term.title()} Cohort {nm}')
            ax.legend()

        for model_term, C in B.groupby('model_term'):
            label = model_term+' model'
            ax = axes[i,0]
            RocCurveDisplay.from_predictions(C['actual'], C['prediction'], ax=ax, name=label)
            ax.plot([0, 1], [0, 1], 'k--')
            fmt('ROC')
            
            ax = axes[i,1]
            C['y'] = C['actual'].rolling(window=100, center=True).mean()
            # x, y = C[['prediction', 'y']].sample(frac=0.1).values.T
            x, y = C[['prediction', 'y']].values[::10].T
            ax.plot(x, y, label=label, linestyle='', marker='.', markersize=4)
            ax.plot([x.min(), x.max()], [x.min(), x.max()], 'k-')
            ax.set_xlabel('Probability')
            ax.set_ylabel('Proportion')
            fmt('Calibration')

            # ax = axes[i,2]
            # CalibrationDisplay.from_predictions(C['actual'], C['prediction'], ax=ax, name=label, n_bins=100)#, strategy='quantile')
            # fmt('Calibration')

    fig.suptitle(f'{LAG} term retention'.title())
    plt.tight_layout()
    plt.show()
run_viz()

In [0]:
ipython-autotime==0.3.2
codetiming==1.4.0
openpyxl==3.1.5
numpy==1.26.4
pandas==2.3.1
geopandas==1.1.1
pgeocode==0.5.0
scikit-learn==1.7.0
miceforest==6.0.3
flaml[automl]==2.3.5